In [ ]:
from phi.agent import Agent
from phi.model.openai import OpenAIChat
from phi.tools import Toolkit
import requests

# --- Define a simple Weather Tool ---
import requests

API_KEY = "" 
BASE_URL = "http://api.openweathermap.org/data/2.5/weather"

from phi.agent import Agent
from phi.model.openai import OpenAIChat
import requests

# --- Define a Weather API tool ---
def find_weather(city: str) -> dict:
    """Fetch current weather for a given city using OpenWeatherMap API."""
    API_KEY = ""  
    BASE_URL = "http://api.openweathermap.org/data/2.5/weather"

    params = {"q": city, "appid": API_KEY, "units": "metric"}
    try:
        response = requests.get(BASE_URL, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()

        # Normalize output for downstream agents
        return {
            "location": {
                "name": data.get("name"),
                "country": data.get("sys", {}).get("country"),
            },
            "current": {
                "temp_c": data.get("main", {}).get("temp"),
                "feelslike_c": data.get("main", {}).get("feels_like"),
                "condition": data.get("weather", [{}])[0].get("description"),
                "wind_kph": data.get("wind", {}).get("speed"),
                "humidity": data.get("main", {}).get("humidity"),
            },
        }
    except Exception as e:
        return {"error": str(e)}

# --- Define Agent 1 ---
weather_agent = Agent(
    name="Weather Agent",
    model=OpenAIChat(id="gpt-4o-mini"),
    tools=[find_weather],   # attach the weather API tool
    show_tool_calls=True,
    description="Takes user input and calls the Weather API to retrieve weather data",
    instructions=["Always respond with normalized JSON containing location and current weather details"],
    markdown=True,
)



In [8]:
from phi.agent import Agent
from phi.model.openai import OpenAIChat

# --- Define a simple parser tool ---
def summarize_weather(weather: dict) -> str:
    """Convert weather JSON into a readable paragraph."""
    if "error" in weather:
        return f"Unable to retrieve weather: {weather['error']}"

    loc = weather.get("location", {})
    cur = weather.get("current", {})

    city = loc.get("name", "Unknown location")
    region = loc.get("region", "")
    country = loc.get("country", "")
    time = loc.get("localtime", "")
    condition = cur.get("condition", "N/A")
    temp = cur.get("temp_c", "N/A")
    feels = cur.get("feelslike_c", "N/A")
    wind = cur.get("wind_kph", "N/A")
    humidity = cur.get("humidity", "N/A")
    uv = cur.get("uv", "N/A")

    return (
        f"Weather update for {city} ({region}, {country}) at {time}: "
        f"{condition}. The temperature is {temp}°C, feels like {feels}°C. "
        f"Wind speed is {wind} kph, humidity is {humidity}%, and the UV index is {uv}."
    )

# --- Define Agent 2 ---
content_agent = Agent(
    name="Content Agent",
    model=OpenAIChat(id="gpt-4o-mini"),
    tools=[summarize_weather],   # attach the summarizer tool
    show_tool_calls=True,
    description="Parses weather JSON and converts it into a meaningful paragraph",
    instructions=["Always respond with a single concise paragraph"],
    markdown=True,
)


In [10]:
from phi.agent import Agent
from phi.model.openai import OpenAIChat
from fpdf import FPDF

# --- Define a PDF generation tool ---
def generate_pdf(summary: str, filename: str = "weather_report.pdf") -> str:
    """Convert a weather summary paragraph into a PDF file."""
    try:
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", "B", 16)
        pdf.cell(0, 10, "Weather Report", ln=True)
        pdf.ln(10)

        pdf.set_font("Arial", size=12)
        pdf.multi_cell(0, 10, summary)

        pdf.output(filename)
        return f"PDF generated successfully: {filename}"
    except Exception as e:
        return f"Error generating PDF: {e}"

# --- Define Agent 3 ---
pdf_agent = Agent(
    name="PDF Agent",
    model=OpenAIChat(id="gpt-4o-mini"),
    tools=[generate_pdf],   # attach the PDF tool
    show_tool_calls=True,
    description="Converts the generated weather summary paragraph into a PDF file",
    instructions=["Always confirm PDF creation and return the filename"],
    markdown=True,
)


In [13]:
import streamlit as st

st.set_page_config(page_title="Weather Report Generator", page_icon="⛅")
st.title("Multi-Agent Weather Reporter")

city = st.text_input("Enter a city name", "Hyderabad")
run_button = st.button("Generate Report")

if run_button and city.strip():
    with st.spinner("Fetching weather and generating report..."):
        # Step 1: Weather Agent
        weather_result = weather_agent.run(f"Get weather for {city}")
        weather_json = weather_result.content

        # Step 2: Content Agent
        content_result = content_agent.run(weather_json)
        summary_text = content_result.content

        # Step 3: PDF Agent
        pdf_result = pdf_agent.run(summary_text)
        pdf_filename = pdf_result.content if isinstance(pdf_result.content, str) else "weather_report.pdf"

    st.subheader("Weather Summary")
    st.write(summary_text)

    st.subheader("Download PDF")
    try:
        with open(pdf_filename, "rb") as f:
            st.download_button(
                label="Download Weather Report PDF",
                data=f,
                file_name=pdf_filename,
                mime="application/pdf",
            )
    except Exception as e:
        st.error(f"Could not read the PDF: {e}")

2026-01-05 10:37:36.769 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-05 10:37:36.770 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-05 10:37:36.834 
  command:

    streamlit run C:\Users\prasa\AppData\Roaming\Python\Python313\site-packages\ipykernel_launcher.py [ARGUMENTS]
2026-01-05 10:37:36.834 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-05 10:37:36.835 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-05 10:37:36.835 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-05 10:37:36.836 Thread 'MainThread': missing ScriptRunContext! This warning can b